In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import math
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from scipy import stats
from collections import Counter

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.linear_model import LogisticRegression

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import the train file from download
train = pd.read_csv("./sample_data/train.csv")
test = pd.read_csv("./sample_data/test.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
#Drop the column from the train

X_train = train.drop(columns = ['Survived'])
Y_train = train['Survived']

combined = pd.concat([X_train, test])
combined = combined.dropna(subset=["Age"])

combined['Age Group'] = pd.cut(combined['Age'], bins=[0, 18, 35, 100], labels=[1, 2, 3])
combined

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age Group
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,3
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,2
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,2
...,...,...,...,...,...,...,...,...,...,...,...,...
409,1301,3,"Peacock, Miss. Treasteall",female,3.0,1,1,SOTON/O.Q. 3101315,13.7750,NaN,S,1
411,1303,1,"Minahan, Mrs. William Edward (Lillian E Thorpe)",female,37.0,1,0,19928,90.0000,C78,Q,3
412,1304,3,"Henriksson, Miss. Jenny Lovisa",female,28.0,0,0,347086,7.7750,NaN,S,2
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,3


In [ ]:
X_age = combined[['Pclass', 'Sex', 'SibSp', 'Parch']]
X_age = pd.get_dummies(X_age, columns = ['Sex'], dtype = int)
Y_age = combined['Age Group'].values

model = LogisticRegression()
folds = 10
kf = KFold(n_splits=folds, shuffle=True)
scores = cross_val_score(model, X_age, Y_age, cv=kf, scoring='accuracy')

# Print the accuracy scores for each fold and the mean accuracy
for i, score in enumerate(scores, 1):
    print(f'Fold {i}: {score:.2f}')

mean_accuracy = scores.mean()
print(f'Mean Accuracy: {mean_accuracy:.2f}')

Fold 1: 0.52
Fold 2: 0.63
Fold 3: 0.61
Fold 4: 0.50
Fold 5: 0.68
Fold 6: 0.54
Fold 7: 0.62
Fold 8: 0.56
Fold 9: 0.59
Fold 10: 0.63
Mean Accuracy: 0.59


In [ ]:
X_train_age, X_test_age, y_train_age, y_test_age = train_test_split(X_age, Y_age, test_size=0.2, random_state=42)

In [ ]:
# MLP Classifier Build Up for Age
clf = MLPClassifier(hidden_layer_sizes = 18,
                    activation = 'relu',      #Default activation is 'relu'. Also check 'identity','logistic'
                    max_iter = 200,           #Default max_iter is 200
                    solver = 'lbfgs',           #Default solver is 'adam'. Also, check 'lbfgs'.
                    verbose = False,              #Default verbose is False
                    random_state = 27)        #Default n_iter_no_change=10


clf.fit(X_train_age, y_train_age)

# Predict train set
print('Accuracy on training set:')
y_pred_age = clf.predict(X_train_age)
print(accuracy_score(y_train_age, y_pred_age))

Accuracy on training set:
0.6519138755980861


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [ ]:
# Predicting Model Build Up
X_train_pred = X_train.loc[:, ['Pclass', 'Sex', 'Age', 'Fare']]
Y_train_pred = train['Survived']
X_train_pred = pd.get_dummies(X_train_pred, columns = ['Sex'], dtype = int)
#median_age_train = X_train_pred["Age"].median()
#X_train_pred["Age"].fillna(median_age_train, inplace = True)
X_train_pred = X_train_pred.fillna(0)

scalar = StandardScaler()

X_train_pred['Age'] = scalar.fit_transform(np.array(X_train_pred['Age']).reshape(-1, 1))
X_train_pred['Fare'] = scalar.fit_transform(np.array(X_train_pred['Fare']).reshape(-1, 1))

X_train_pred

,Pclass,Age,Fare,Sex_female,Sex_male
0,3,-0.102313,-0.502445,0,1
1,1,0.807492,0.786845,1,0
2,3,0.125138,-0.488854,1,0
3,1,0.636903,0.420730,1,0
4,3,0.636903,-0.486337,0,1
...,...,...,...,...,...
886,2,0.182001,-0.386671,0,1
887,1,-0.272901,-0.044381,1,0
888,3,-1.353294,-0.176263,1,0
889,1,0.125138,-0.044381,0,1


In [ ]:
Y_train_pred

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [ ]:
# MLP Classifier Build Up
clf = MLPClassifier(hidden_layer_sizes = 10,
                    activation = 'logistic',      #Default activation is 'relu'. Also check 'identity','logistic'
                    max_iter = 200,           #Default max_iter is 200
                    solver = 'lbfgs',           #Default solver is 'adam'. Also, check 'lbfgs'.
                    verbose = False,              #Default verbose is False
                    random_state = 27)        #Default n_iter_no_change=10


clf.fit(X_train_pred, Y_train_pred)

# Predict train set
print('Accuracy on training set:')
y_pred_train = clf.predict(X_train_pred)
print(accuracy_score(Y_train_pred, y_pred_train))

Accuracy on training set:
0.8417508417508418


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [ ]:
# Prediction for test set
X_test_pred = test.loc[:, ['Pclass', 'Sex', 'Age', 'Fare']]
X_test_pred = pd.get_dummies(X_test_pred, columns = ['Sex'], dtype = int)
median_age_test = X_test_pred["Age"].median()
X_test_pred["Age"].fillna(median_age_test, inplace = True)
X_test_pred = X_test_pred.fillna(0)

scalar = StandardScaler()

X_test_pred['Age'] = scalar.fit_transform(np.array(X_test_pred['Age']).reshape(-1, 1))
X_test_pred['Fare'] = scalar.fit_transform(np.array(X_test_pred['Fare']).reshape(-1, 1))

X_test_pred

,Pclass,Age,Fare,Sex_female,Sex_male
0,3,0.386231,-0.496637,0,1
1,3,1.371370,-0.511497,1,0
2,2,2.553537,-0.463335,0,1
3,3,-0.204852,-0.481704,0,1
4,3,-0.598908,-0.416740,1,0
...,...,...,...,...,...
413,3,-0.204852,-0.492680,0,1
414,1,0.740881,1.314641,1,0
415,3,0.701476,-0.507017,0,1
416,3,-0.204852,-0.492680,0,1


In [ ]:
y_pred_test = clf.predict(X_test_pred)
y_pred_test

d = {"PassengerId": test['PassengerId'], "Survived": y_pred_test}
prediction = pd.DataFrame(data = d)
prediction

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
out = pd.read_csv('./predictions7.csv')

equal_count = (out['Survived'] == prediction["Survived"]).sum()
equal_count

397

In [ ]:
test1 = pd.read_csv('./gender_submission.csv')
test2 = pd.read_csv('./test_submission5.csv')
test3 = pd.read_csv('./predictions7.csv')

result_array = [test1["Survived"].values, test2["Survived"].values, test3["Survived"].values]
transposed_data = np.array(result_array).T

majority_values = []

for column in transposed_data:
    counts = Counter(column)
    majority_value, majority_count = counts.most_common(1)[0]
    majority_values.append(majority_value)

d = {"PassengerId": test['PassengerId'], "Survived": majority_values}
prediction = pd.DataFrame(data = d)
prediction

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
# Export csv
os.makedirs('DMA23-lab5/test_pred', exist_ok=True)
prediction.to_csv('DMA23-lab5/test_pred/predictions9.csv', index = False)